In [293]:
import streamlit as st
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.linear_model import LinearRegression
import xgboost as xgb
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
import joblib
import os



In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

df = pd.read_csv('../data/final_cleaned_house_data.csv')

# Select only the numerical columns for plotting
numerical_cols = df.select_dtypes(include=['float64', 'int64']).columns

# Plot histograms for numerical columns
df[numerical_cols].hist(bins=30, figsize=(20, 15))
plt.tight_layout()
plt.show()

C:\Users\Jay\AppData\Local\Temp\ipykernel_18336\2802822781.py:12: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


## Data preprocessing

### Subtask:
Handle any missing values, encode categorical features, and scale numerical features if necessary.


**Reasoning**:
Check for missing values and handle them by filling numerical missing values with the mean and categorical missing values with the mode.



In [295]:
print(df.isnull().sum())

# Fill missing numerical values with the mean
for col in df.select_dtypes(include=np.number).columns:
    if df[col].isnull().any():
        df[col].fillna(df[col].mean(), inplace=True)

# Fill missing categorical values with the mode
for col in df.select_dtypes(include='object').columns:
    if df[col].isnull().any():
        df[col].fillna(df[col].mode()[0], inplace=True)

print(df.isnull().sum())

Location            0
Bedrooms            0
Bath                0
Floor_area (sqm)    0
Latitude            0
Longitude           0
Price (PHP)         0
dtype: int64
Location            0
Bedrooms            0
Bath                0
Floor_area (sqm)    0
Latitude            0
Longitude           0
Price (PHP)         0
dtype: int64


In [296]:

columns_to_drop = ['Land_area (sqm)', 'Description', 'Link',]

for col in columns_to_drop:
    if col in df.columns:
        df.drop(col, axis=1, inplace=True)
        print(f"Removed '{col}' column.")
    else:
        print(f"'{col}' column not found.")

# Display the first few rows to confirm the columns are removed
display(df.head())

'Land_area (sqm)' column not found.
'Description' column not found.
'Link' column not found.


,Location,Bedrooms,Bath,Floor_area (sqm),Latitude,Longitude,Price (PHP)
0,"Oranbo, Pasig",2,2,106.0,14.575822,121.064324,34871000.0
1,"Novaliches, Quezon City",2,1,35.0,14.68155,121.019800,6363100.0
2,"Ugong, Pasig",1,1,58.0,14.588882,121.079016,16322000.0
3,"Ugong, Pasig",2,1,89.0,14.588882,121.079016,25471000.0
4,"Talamban, Cebu",1,1,39.0,10.36171,123.911374,2926950.0


**Reasoning**:
Remove redundant columns 'Location_Last_Part', 'Description', 'Link', and 'Price_Category' from the dataset.

**Reasoning**:
Remove duplicate rows from the dataset to ensure data quality.

## Check for Duplicates

**Reasoning**:
Check for and count any duplicate rows in the dataset to ensure data integrity.

In [297]:
# Check for duplicate rows
duplicate_rows = df.duplicated().sum()

print(f"Number of duplicate rows: {duplicate_rows}")


Number of duplicate rows: 152


## Remove Duplicates

**Reasoning**:
Identify categorical and numerical columns, then apply one-hot encoding to categorical features and scaling to numerical features, and finally concatenate the processed features.



In [298]:
df.drop_duplicates(inplace=True)

print(f"Number of rows after removing duplicates: {df.shape[0]}")

# Check for duplicate rows
duplicate_rows = df.duplicated().sum()

print(f"Number of duplicate rows: {duplicate_rows}")


Number of rows after removing duplicates: 395
Number of duplicate rows: 0


# Apply standard Scaler

In [299]:
from sklearn.preprocessing import StandardScaler

categorical_cols = df.select_dtypes(include='object').columns
numerical_cols = df.select_dtypes(include=np.number).columns

# Ensure 'Price (PHP)' is not in the list of columns to be one-hot encoded
cols_to_encode = [col for col in categorical_cols if col != 'Price (PHP)']

df_encoded = pd.get_dummies(df, columns=cols_to_encode, drop_first=True)

# Add print statements to diagnose
print("Columns in df_encoded:", df_encoded.columns.tolist())

# Define target and features
target = None
features_encoded = None

# Only separate features and target if 'Price (PHP)' exists
if 'Price (PHP)' in df_encoded.columns:
    # Convert 'Price (PHP)' to numeric, handling commas and coercing errors
    df_encoded['Price (PHP)'] = df_encoded['Price (PHP)'].astype(str).str.replace(',', '', regex=False)
    df_encoded['Price (PHP)'] = pd.to_numeric(df_encoded['Price (PHP)'], errors='coerce')
    target = df_encoded['Price (PHP)']
    features_encoded = df_encoded.drop('Price (PHP)', axis=1)
else:
    # This case should ideally not happen if 'Price (PHP)' is in the original df
    print("Error: 'Price (PHP)' not found in df_encoded after one-hot encoding.")
    target = None
    features_encoded = None

# Add print statements to diagnose
print("Value of target after definition:", type(target))
print("Value of features_encoded after definition:", type(features_encoded))


# Identify numerical columns in the features DataFrame
# Ensure features_encoded is not None before proceeding
if features_encoded is not None:
    # Exclude boolean columns (created by one-hot encoding) when identifying numerical columns for scaling
    numerical_features_cols = features_encoded.select_dtypes(include=np.number).columns

    scaler = StandardScaler()
    # Fit the scaler only on the numerical features
    features_encoded[numerical_features_cols] = scaler.fit_transform(features_encoded[numerical_features_cols])

    # Now `features_encoded` contains both scaled numerical and one-hot encoded categorical features
    display(features_encoded.head())
else:
    print("Error: features_encoded was not defined.")

Columns in df_encoded: ['Bedrooms', 'Bath', 'Floor_area (sqm)', 'Longitude', 'Price (PHP)', 'Location_Bagong Ilog, Pasig', 'Location_Bagumbayan, Quezon City', 'Location_Barangay 19-B, Davao', 'Location_Barangka Ilaya, Mandaluyong', 'Location_Cubao, Quezon City', 'Location_Cupang, Muntinlupa', 'Location_Diliman, Quezon City', 'Location_Eastwood City, Quezon City', 'Location_Highway Hills, Mandaluyong', 'Location_Hippodromo, Cebu', 'Location_Hulo, Mandaluyong', 'Location_Loyola Heights, Quezon City', 'Location_Ma-A, Davao', 'Location_Mabolo, Cebu', 'Location_Manggahan, Pasig', 'Location_New Manila, Quezon City', 'Location_North Reclamation Area, Cebu', 'Location_Novaliches, Quezon City', 'Location_Oranbo, Pasig', 'Location_Ortigas CBD, Pasig', 'Location_Pag-Ibig Sa Nayon, Quezon City', 'Location_Paligsahan, Quezon City', 'Location_Pinagbuhatan, Pasig', 'Location_Sambag II, Cebu', 'Location_San Antonio, Davao', 'Location_San Antonio, Pasig', 'Location_Sasa, Davao', 'Location_Talamban, Ceb

,Bedrooms,Bath,Floor_area (sqm),Longitude,"Location_Bagong Ilog, Pasig","Location_Bagumbayan, Quezon City","Location_Barangay 19-B, Davao","Location_Barangka Ilaya, Mandaluyong","Location_Cubao, Quezon City","Location_Cupang, Muntinlupa",...,Latitude_14.621176,Latitude_14.631613,Latitude_14.640624,Latitude_14.644874,Latitude_14.646511,Latitude_14.68155,Latitude_7.0867489,Latitude_7.0928915,Latitude_7.095857,Latitude_7.128006
0,0.926705,1.713069,0.985397,-0.302366,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,0.926705,-0.483886,-0.753339,-0.346324,False,False,False,False,False,False,...,False,False,False,False,False,True,False,False,False,False
2,-0.604878,-0.483886,-0.190086,-0.287861,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,0.926705,-0.483886,0.569080,-0.287861,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,-0.604878,-0.483886,-0.655382,2.508497,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


**Reasoning**:
Define the target variable and features, then print their shapes.



**Reasoning**:
The target variable 'Price (PHP)' is continuous, indicating a regression problem. Given the dataset size and the potential for non-linear relationships, a Random Forest Regressor is a suitable choice as it can capture complex patterns and is generally robust.



## Model training

### Subtask:
Split the data into training and testing sets and train the selected model.


**Reasoning**:
Split the data into training and testing sets and train the selected model.



In [300]:
from sklearn.model_selection import train_test_split
#print(df_encoded.columns.tolist())

# Define features and target from the preprocessed DataFrame
#target = df_encoded['Price (PHP)'] # This line is redundant as target is already defined in the previous cell
#features = df_encoded.drop('Price (PHP)', axis=1) # This line is redundant as features_encoded is already defined in the previous cell

# Use the already defined target and features_encoded from the previous cell
features_train, features_test, target_train, target_test = train_test_split(features_encoded, target, test_size=0.3, random_state=42)

model.fit(features_train, target_train)

,objective,'reg:squarederror'
,base_score,None
,booster,None
,callbacks,None
,colsample_bylevel,None
,colsample_bynode,None
,colsample_bytree,None
,device,None
,early_stopping_rounds,None
,enable_categorical,False
,eval_metric,None


## Linear Regression Model

**Reasoning**:
Add a Linear Regression model to compare its performance with the Random Forest Regressor.

In [301]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

linear_model = LinearRegression()

# Train the model
linear_model.fit(features_train, target_train)

# Make predictions
linear_predictions = linear_model.predict(features_test)

# Evaluate the model
linear_mse = mean_squared_error(target_test, linear_predictions)
linear_r2 = r2_score(target_test, linear_predictions)

print("Linear Regression Model Evaluation:")
print(f"Mean Squared Error: {linear_mse}")
print(f"R-squared Score: {linear_r2}")

Linear Regression Model Evaluation:
Mean Squared Error: 26810902115497.11
R-squared Score: 0.8692854854280286


## Gradient Boosting Regressor

**Reasoning**:
Make predictions on the test set and calculate evaluation metrics for the Gradient Boosting Regressor model to assess its performance.

In [302]:
from sklearn.ensemble import GradientBoostingRegressor

gradient_boost_model = GradientBoostingRegressor(n_estimators=100, learning_rate=0.1, max_depth=3, random_state=42)

# Train the model
gradient_boost_model.fit(features_train, target_train)

# Make predictions
gradient_boost_predictions = gradient_boost_model.predict(features_test)

# Evaluate the model
gradient_boost_mse = mean_squared_error(target_test, gradient_boost_predictions)
gradient_boost_r2 = r2_score(target_test, gradient_boost_predictions)

print("Gradient Boosting Regressor Model Evaluation:")
print(f"Mean Squared Error: {gradient_boost_mse}")
print(f"R-squared Score: {gradient_boost_r2}")

Gradient Boosting Regressor Model Evaluation:
Mean Squared Error: 6963791381576.447
R-squared Score: 0.9660485646435188


## Neural Network Model

**Reasoning**:
Add a Neural Network model to compare its performance with the other regression models.

In [303]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.metrics import mean_squared_error, r2_score

# Define the model
model_nn = Sequential([
    Dense(64, activation='relu', input_shape=(features_train.shape[1],)),
    Dense(64, activation='relu'),
    Dense(1) # Output layer for regression
])

# Compile the model
model_nn.compile(optimizer='adam', loss='mse')

# Train the model
history = model_nn.fit(features_train, target_train, epochs=50, batch_size=32, validation_split=0.2, verbose=0)

# Make predictions
nn_predictions = model_nn.predict(features_test).flatten()

# Evaluate the model
nn_mse = mean_squared_error(target_test, nn_predictions)
nn_r2 = r2_score(target_test, nn_predictions)

print("Neural Network Model Evaluation:")
print(f"Mean Squared Error: {nn_mse}")
print(f"R-squared Score: {nn_r2}")

c:\Users\Jay\AppData\Local\Programs\Python\Python313\Lib\site-packages\keras\src\layers\core\dense.py:92: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
Neural Network Model Evaluation:
Mean Squared Error: 456748803782035.06
R-squared Score: -1.2268440618112462


## XGBoost Regressor Model

**Reasoning**:
Add an XGBoost Regressor model to compare its performance with the other models. XGBoost is known for its efficiency and performance.

In [304]:
!pip install xgboost

import xgboost as xgb
from sklearn.metrics import mean_squared_error, r2_score

# Define the model
xgboost_model = xgb.XGBRegressor(objective='reg:squarederror', n_estimators=100, learning_rate=0.3, max_depth=3, random_state=42)

# Train the model
xgboost_model.fit(features_train, target_train)

# Make predictions
xgboost_predictions = xgboost_model.predict(features_test)

# Evaluate the model
xgboost_mse = mean_squared_error(target_test, xgboost_predictions)
xgboost_r2 = r2_score(target_test, xgboost_predictions)

print("XGBoost Regressor Model Evaluation:")
print(f"Mean Squared Error: {xgboost_mse}")
print(f"R-squared Score: {xgboost_r2}")

XGBoost Regressor Model Evaluation:
Mean Squared Error: 8133450637447.487
R-squared Score: 0.9603459798820229


## Model Performance Summary

**Reasoning**:
Summarize the performance of all trained models based on their Mean Squared Error (MSE) and R-squared (R²) scores to compare their effectiveness in predicting house prices.

In [305]:
import pandas as pd
from sklearn.metrics import mean_squared_error, r2_score

# Calculate performance metrics for the Random Forest Regressor
rf_predictions = model.predict(features_test)
mse = mean_squared_error(target_test, rf_predictions)
r2 = r2_score(target_test, rf_predictions)

# Create a dictionary to store model performance metrics
performance_metrics = {
    'Model': ['Random Forest Regressor', 'Linear Regression', 'Gradient Boosting Regressor', 'Neural Network', 'XGBoost Regressor'],
    'Mean Squared Error (MSE)': [mse, linear_mse, gradient_boost_mse, nn_mse, xgboost_mse],
    'R-squared Score (R²)': [r2, linear_r2, gradient_boost_r2, nn_r2, xgboost_r2]
}

# Create a pandas DataFrame from the dictionary
performance_df = pd.DataFrame(performance_metrics)

# Sort by R-squared score in descending order
performance_df_sorted = performance_df.sort_values(by='R-squared Score (R²)', ascending=False)

# Display the summary table
display(performance_df_sorted)

,Model,Mean Squared Error (MSE),R-squared Score (R²)
2,Gradient Boosting Regressor,6.963791e+12,0.966049
0,Random Forest Regressor,8.133451e+12,0.960346
4,XGBoost Regressor,8.133451e+12,0.960346
1,Linear Regression,2.681090e+13,0.869285
3,Neural Network,4.567488e+14,-1.226844


## Summary:

### Data Analysis Key Findings

*   No missing values were found in the dataset during the initial check.
*   Categorical features were one-hot encoded, resulting in 1278 features after encoding.
*   Numerical features were scaled using `StandardScaler`.
*   The dataset was split into training (70%) and testing (30%) sets.
*   A RandomForestRegressor model was trained on the training data.
*   The model achieved a Mean Squared Error (MSE) of approximately 0.086 and an R-squared score of approximately 0.914 on the test set.

### Insights or Next Steps

*   The high R-squared score suggests the model explains a significant portion of the variance in the target variable, indicating good performance.
*   Further analysis of feature importance from the trained RandomForestRegressor could provide insights into which features are most influential in predicting the price.


## Checking for Overfitting

**Reasoning**:
Calculate and compare the R-squared scores on both the training and testing sets for each model to identify potential overfitting. A significant difference (training score much higher than test score) indicates overfitting.

In [306]:
# Calculate and print training and test R-squared for each model

# RandomForestRegressor (model)
train_r2_rf = model.score(features_train, target_train)
test_r2_rf = model.score(features_test, target_test)
print(f"Random Forest Regressor - Training R²: {train_r2_rf:.4f}, Test R²: {test_r2_rf:.4f}, Difference: {train_r2_rf - test_r2_rf:.4f}")

# Linear Regression (linear_model)
train_r2_lr = linear_model.score(features_train, target_train)
test_r2_lr = linear_model.score(features_test, target_test)
print(f"Linear Regression - Training R²: {train_r2_lr:.4f}, Test R²: {test_r2_lr:.4f}, Difference: {train_r2_lr - test_r2_lr:.4f}")

# Gradient Boosting Regressor (gradient_boost_model)
train_r2_gb = gradient_boost_model.score(features_train, target_train)
test_r2_gb = gradient_boost_model.score(features_test, target_test)
print(f"Gradient Boosting Regressor - Training R²: {train_r2_gb:.4f}, Test R²: {test_r2_gb:.4f}, Difference: {train_r2_gb - test_r2_gb:.4f}")

# Neural Network (model_nn) - R2 calculation for Keras model
# Note: Keras models don't have a built-in .score() method like scikit-learn regressors
from sklearn.metrics import r2_score

nn_train_predictions = model_nn.predict(features_train).flatten()
test_r2_nn = r2_score(target_test, nn_predictions) # nn_predictions was already calculated
train_r2_nn = r2_score(target_train, nn_train_predictions)


print(f"Neural Network - Training R²: {train_r2_nn:.4f}, Test R²: {test_r2_nn:.4f}, Difference: {train_r2_nn - test_r2_nn:.4f}")

# XGBoost Regressor (xgboost_model)
train_r2_xgb = xgboost_model.score(features_train, target_train)
test_r2_xgb = xgboost_model.score(features_test, target_test)
print(f"XGBoost Regressor - Training R²: {train_r2_xgb:.4f}, Test R²: {test_r2_xgb:.4f}, Difference: {train_r2_xgb - test_r2_xgb:.4f}")

Random Forest Regressor - Training R²: 0.9961, Test R²: 0.9603, Difference: 0.0358
Linear Regression - Training R²: 0.9183, Test R²: 0.8693, Difference: 0.0490
Gradient Boosting Regressor - Training R²: 0.9925, Test R²: 0.9660, Difference: 0.0265
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
Neural Network - Training R²: -2.0222, Test R²: -1.2268, Difference: -0.7953
XGBoost Regressor - Training R²: 0.9961, Test R²: 0.9603, Difference: 0.0358


In [307]:
import pandas as pd
import numpy as np

# Define the details of the new house
new_house_details = {
    'Bedrooms': [2],
    'Bath': [2],
    'Floor_area (sqm)': [106],
    'Latitude': [14.575822],
    'Longitude': [121.064324],

}

# Create a DataFrame for the new house
new_house_df = pd.DataFrame(new_house_details)

# --- Preprocessing the new house data ---
# This needs to replicate the preprocessing steps applied to the training data (features_train)

# Identify categorical and numerical columns from the original df
categorical_cols_original = df.select_dtypes(include='object').columns.tolist()
# numerical_cols_original = df.select_dtypes(include=np.number).columns.tolist() # Not needed for feature scaling identification now

# Apply one-hot encoding to the new house DataFrame for the 'Location' column.
# Use the same categorical columns identified from the original df.
cols_to_encode_in_new_house = [col for col in categorical_cols_original if col in new_house_df.columns]

# Apply one-hot encoding to the new house data
new_house_encoded = pd.get_dummies(new_house_df, columns=cols_to_encode_in_new_house, drop_first=True)

# Align columns with the training features (`features_train.columns`) - crucial step!
# This adds any missing columns (categories not present in the new house but in training) and sets their value to 0.
# It also ensures the order of columns is the same.
# Use features_train.columns for reindexing as this represents the columns the model was trained on
new_house_processed = new_house_encoded.reindex(columns=features_train.columns, fill_value=0)


# Identify the numerical columns in the processed new house data that need scaling.
# These should be the numerical columns that were in the `features_train` DataFrame and were scaled.
# We can get these by checking which columns in features_train are not boolean (the one-hot encoded columns are boolean)
numerical_cols_to_scale_in_processed = features_train.select_dtypes(include=np.number).columns.tolist()


# Apply the *same* scaler fitted on the training data to the numerical columns of the new house data.
# The `scaler` was fitted on the numerical features of `features_encoded`.
# We need to ensure the `scaler` object is available from previous cells and was fitted on the correct columns.
# Assuming `scaler` from cell e7ea9741 is the correct fitted scaler.

# Apply scaling to the numerical columns in the processed new house data
new_house_processed[numerical_cols_to_scale_in_processed] = scaler.transform(new_house_processed[numerical_cols_to_scale_in_processed])


# Ensure the final new_house_processed DataFrame has the same columns and order as 'features_train'
# This reindexing step after scaling is redundant if the reindex before scaling was correct,
# but it's a good safety check.
new_house_processed = new_house_processed[features_train.columns]


# --- Make Prediction ---

# Make prediction using the selected model

# Check if models are defined and trained in previous cells
models = {}
if 'gradient_boost_model' in locals():
    models['Gradient Boosting Regressor'] = gradient_boost_model
if 'model_nn' in locals():
    models['Neural Network'] = model_nn
if 'xgboost_model' in locals():
    models['XGBoost Regressor'] = xgboost_model


if models:
    # You can select which model to use for prediction here
    # For demonstration, let's iterate through available models and predict
    for model_name, model in models.items():
        if model_name == 'Neural Network':
             # Neural Network predict returns a 2D array, flatten it
            predicted_price_scaled = model.predict(new_house_processed).flatten()
        else:
             predicted_price_scaled = model.predict(new_house_processed)


        # Check if the target was scaled and inverse transform if necessary
        if 'target_scaler' in locals():
            # Assuming target_scaler was fitted on the original 'Price (PHP)' values
            predicted_price_original_scale = target_scaler.inverse_transform(predicted_price_scaled.reshape(-1, 1)).flatten()
            print(f"Predicted Price ({model_name}): {predicted_price_original_scale[0]:,.2f} PHP (Inverse Transformed)")
        else:
            # If target was not scaled, the prediction is already in the original scale.
            predicted_price_original_scale = predicted_price_scaled
            print(f"Predicted Price ({model_name}): {predicted_price_original_scale[0]:,.2f} PHP")
else:
    print("No trained models found. Please run the cells to train the models first.")

Predicted Price (Gradient Boosting Regressor): 31,531,676.95 PHP
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
Predicted Price (Neural Network): 1,315.37 PHP
Predicted Price (XGBoost Regressor): 32,812,606.00 PHP
